
Superconducting Materials Design with VAE + Random Forest + Nearest-Neighbor Mapping

- Trains a RandomForestRegressor to predict critical temperature (Tc) from features.
- Trains a VAE on the same feature space to learn a latent distribution.
- Generates new candidate feature vectors using the VAE.
- Predicts Tc of generated candidates with the Random Forest.
- Maps generated candidates to the closest known compositions in unique_m.csv.



In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.spatial.distance import cdist

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [6]:
# -----------------------------
# Paths
# -----------------------------
TRAIN_CSV = "train.csv"
UNIQUE_M_CSV = "unique_m.csv"
OUT_GENERATED_CSV = "generated_candidates_with_matches.csv"

In [7]:
# -----------------------------
# Load and preprocess data
# -----------------------------
train_df = pd.read_csv(TRAIN_CSV)
features = train_df.drop(columns=["critical_temp"])
target = train_df["critical_temp"]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# -----------------------------
# Train / validation split
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(features_scaled, target, test_size=0.2, random_state=42)


In [8]:
# -----------------------------
# Random Forest regressor (Tc prediction)
# -----------------------------
regressor = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1)
regressor.fit(X_train, y_train)

y_val_pred = regressor.predict(X_val)
mse = mean_squared_error(y_val, y_val_pred)
rmse = mse ** 0.5
r2 = r2_score(y_val, y_val_pred)

print(f"[RandomForest] Validation MSE : {mse:.3f}")
print(f"[RandomForest] Validation RMSE: {rmse:.3f}")
print(f"[RandomForest] Validation R^2 : {r2:.3f}")

[RandomForest] Validation MSE : 80.643
[RandomForest] Validation RMSE: 8.980
[RandomForest] Validation R^2 : 0.930


In [9]:
# -----------------------------
# VAE model (unconditional VAE)
# -----------------------------
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=10):
        super(VAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
        )
        self.mu = nn.Linear(32, latent_dim)
        self.log_var = nn.Linear(32, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        encoded = self.encoder(x)
        mu = self.mu(encoded)
        log_var = self.log_var(encoded)
        z = self.reparameterize(mu, log_var)
        decoded = self.decoder(z)
        return decoded, mu, log_var


def vae_loss(recon_x, x, mu, log_var, beta=0.1):
    # Reconstruction loss (MSE)
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction="mean")
    # Standard KL divergence term
    kl_loss = -0.5 * torch.mean(1 + log_var - mu.pow(2) - log_var.exp())
    return recon_loss + beta * kl_loss, recon_loss, kl_loss


input_dim = features_scaled.shape[1]
latent_dim = 10
vae = VAE(input_dim, latent_dim)

In [10]:
# -----------------------------
# VAE training setup
# -----------------------------
tensor_data = torch.tensor(features_scaled, dtype=torch.float32)
dataset = TensorDataset(tensor_data)
data_loader = DataLoader(dataset, batch_size=256, shuffle=True)

optimizer = optim.Adam(vae.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

num_epochs = 100
for epoch in range(num_epochs):
    vae.train()
    total_loss = 0.0
    total_recon = 0.0
    total_kl = 0.0

    for (batch_x,) in data_loader:
        optimizer.zero_grad()
        recon_x, mu, log_var = vae(batch_x)
        loss, recon_loss, kl_loss = vae_loss(recon_x, batch_x, mu, log_var, beta=0.1)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_recon += recon_loss.item()
        total_kl += kl_loss.item()

    scheduler.step()
    avg_loss = total_loss / len(data_loader)
    avg_recon = total_recon / len(data_loader)
    avg_kl = total_kl / len(data_loader)

    if epoch % 10 == 0 or epoch == num_epochs - 1:
        print(
            f"[VAE] Epoch {epoch+1}/{num_epochs} "
            f"Loss: {avg_loss:.4f}, Recon: {avg_recon:.4f}, KL: {avg_kl:.4f}"
        )

[VAE] Epoch 1/100 Loss: 0.8022, Recon: 0.7787, KL: 0.2352
[VAE] Epoch 11/100 Loss: 0.2657, Recon: 0.1849, KL: 0.8078
[VAE] Epoch 21/100 Loss: 0.2162, Recon: 0.1338, KL: 0.8237
[VAE] Epoch 31/100 Loss: 0.2043, Recon: 0.1223, KL: 0.8205
[VAE] Epoch 41/100 Loss: 0.1961, Recon: 0.1143, KL: 0.8176
[VAE] Epoch 51/100 Loss: 0.1941, Recon: 0.1114, KL: 0.8275
[VAE] Epoch 61/100 Loss: 0.1891, Recon: 0.1071, KL: 0.8205
[VAE] Epoch 71/100 Loss: 0.1871, Recon: 0.1052, KL: 0.8183
[VAE] Epoch 81/100 Loss: 0.1871, Recon: 0.1050, KL: 0.8207
[VAE] Epoch 91/100 Loss: 0.1844, Recon: 0.1025, KL: 0.8186
[VAE] Epoch 100/100 Loss: 0.1841, Recon: 0.1019, KL: 0.8218


In [11]:
# -----------------------------
# Generate new candidates
# -----------------------------
def generate_new_materials(n_samples=50, z_scale=1.0):
    """
    Sample from the VAE latent space and return:
      - generated feature vectors in original scaling
      - predicted Tc from the RandomForest regressor
    """
    vae.eval()
    with torch.no_grad():
        z = torch.randn(n_samples, latent_dim) * z_scale
        generated_scaled = vae.decoder(z).detach().cpu().numpy()
    # Inverse transform to original feature scale
    generated_features = scaler.inverse_transform(generated_scaled)
    generated_df = pd.DataFrame(generated_features, columns=features.columns)
    generated_df["Predicted_Tc"] = regressor.predict(generated_scaled)
    return generated_df


n_samples = 100  # adjust as needed
generated_df = generate_new_materials(n_samples=n_samples, z_scale=1.0)

# Optionally filter candidates by high predicted Tc
HIGH_TC_THRESHOLD = 80.0  # K, for example
high_tc_df = generated_df[generated_df["Predicted_Tc"] >= HIGH_TC_THRESHOLD].copy()
high_tc_df.reset_index(drop=True, inplace=True)

print(f"Generated {len(generated_df)} candidates.")
print(f"{len(high_tc_df)} candidates with Predicted_Tc >= {HIGH_TC_THRESHOLD} K.")


Generated 100 candidates.
2 candidates with Predicted_Tc >= 80.0 K.


In [17]:
# -----------------------------
# Map to closest known compositions (unique_m)
# -----------------------------
# The original issue is that `unique_m_df` (from unique_m.csv) contains elemental compositions
# (e.g., 'H', 'He', 'Li'), while `high_tc_df` (generated candidates) contains
# descriptive material properties (e.g., 'number_of_elements', 'mean_atomic_mass').
# This mismatch meant there were no common feature columns for comparison, leading to ValueError.

# To fix this, we need to ensure that the dataset used for `unique_features` has
# the same descriptive feature columns as `high_tc_df`.
# The `train_df` already contains these descriptive features along with `critical_temp`.
# `unique_m_df` (from `unique_m.csv`) contains `material` identifiers and `critical_temp`.

# We will use the descriptive features from `train_df` as our reference for "known compositions"
# for the purpose of distance calculation. We will then retrieve the 'material' and
# 'critical_temp' from `unique_m_df` for the closest matches, leveraging the implicit
# row-wise correspondence between `train_df` and `unique_m_df` (both have 21263 rows).

# Load unique_m.csv to get material names and known critical temperatures.
# We will use its `material` and `critical_temp` columns later.
unique_m_info_df = pd.read_csv(UNIQUE_M_CSV)
unique_m_info_df.columns = unique_m_info_df.columns.str.strip()

# Prepare the reference DataFrame for matching, using descriptive features from train_df.
# `train_df` contains the same descriptive features as `high_tc_df` (excluding 'Predicted_Tc').
# We define the feature columns to use for matching based on the original `features` DataFrame.
feature_columns_for_match = features.columns.tolist() # These are the descriptive feature names

# The 'unique_features' for distance calculation will be the descriptive features from `train_df`.
unique_features = train_df[feature_columns_for_match].values

# The generated candidates from high_tc_df also need to be restricted to these same feature columns
# for consistent comparison.
generated_features_for_match = high_tc_df[feature_columns_for_match].values

# Compute distances and get nearest neighbors
distances = cdist(generated_features_for_match, unique_features, metric="euclidean")
closest_indices = np.argmin(distances, axis=1)

# Retrieve the 'material' and 'critical_temp' for the closest matches.
# We use `unique_m_info_df` for this, assuming its rows correspond to `train_df`'s rows
# based on the matching `closest_indices`.
closest_materials = unique_m_info_df.iloc[closest_indices].reset_index(drop=True)

# Attach matched material and its known Tc
high_tc_df["Closest_Material"] = closest_materials["material"].values
high_tc_df["Known_Tc_Closest"] = closest_materials["critical_temp"].values

# Tc consistency metric
high_tc_df["Tc_Diff"] = high_tc_df["Predicted_Tc"] - high_tc_df["Known_Tc_Closest"]


In [18]:
# -----------------------------
# Save results
# -----------------------------
high_tc_df.to_csv(OUT_GENERATED_CSV, index=False)
print(f"Saved high-Tc candidates with nearest known matches to {OUT_GENERATED_CSV}")

# Preview
print(high_tc_df[["Predicted_Tc", "Closest_Material", "Known_Tc_Closest", "Tc_Diff"]]
      .sort_values(by="Predicted_Tc", ascending=False)
      .head())


Saved high-Tc candidates with nearest known matches to generated_candidates_with_matches.csv
   Predicted_Tc          Closest_Material  Known_Tc_Closest    Tc_Diff
0    100.729026             Hg1Ba2Ca1Cu2O             125.0 -24.270974
1     80.406248  Pb0.7Sr2Ca0.5Y0.5Cu2.3O7              40.0  40.406248
